In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.ensemble import GradientBoostingRegressor
from collections import defaultdict


/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
from importlib import reload
from data_cleaning import data_cleaning as dc
from models import contract_regressor as cr

In [178]:
reload(dc)

<module 'data_cleaning.data_cleaning' from '/Users/colinshanahan/galvanize/nhl-contract-predictions/data_cleaning/data_cleaning.py'>

In [67]:
def clean_FA():
    df = pd.read_csv('../data/free_agents_2019.csv')
    df['signing_status'] = df['ufa_status'].apply(lambda x: 1 if 'UFA' else 0)
    df = df[(df.position != 'G') & (df.position != 27) & (df.position != 30)]
    df['forward'] = (df.position != 'D').replace([True, False], [1, 0])
    df['Season_Player'] = '2018 ' + df['Player']
    df.set_index(df['Season_Player'], inplace=True)
    df.drop('Season_Player', axis=1, inplace=True)
    fa = df[['Player', 'age', 'forward', 'signing_status']]
    fa['signing_year'] = 2019
    return fa

In [102]:
df = dc.clean_FA()
df.sample(5)

,Player,signing_age,forward,signing_status,signing_year
Season_Player,,,,,
2018 Dysin Mayo,Dysin Mayo,22,0,1,2019
2018 Alex Gallant,Alex Gallant,26,1,1,2019
2018 Julius Honka,Julius Honka,23,0,1,2019
2018 Chase Lang,Chase Lang,22,1,1,2019
2018 Michael Downing,Michael Downing,23,0,1,2019


In [103]:
df.shape

(540, 5)

In [105]:
len(df.Player.unique())

539

In [126]:
st = dc.clean_features_data(sql=False, new_fas=True)

In [116]:
st.head(6)

,Player,signing_age,forward,signing_status,signing_year,GP,TOI,Goals,Total Assists,First Assists,...,mean Shots/60,Hits/60,mean Hits/60,PIM/60,mean PIM/60,Penalties Drawn/60,mean Penalties Drawn/60,mean Faceoffs pct,Goalness,mean Goalness
Season_Player,,,,,,,,,,,,,,,,,,,,,
2018 Mitchell Marner,Mitchell Marner,21,1,1,2019,82.0,1625.433333,26.0,68.0,52.0,...,8.485720,1.070484,1.421323,0.812091,1.210235,0.738265,0.830278,0.986111,0.273684,0.297778
2018 Brayden Point,Brayden Point,23,1,1,2019,79.0,1494.600000,41.0,51.0,28.0,...,7.464184,1.244480,1.349457,1.124047,0.927752,0.923324,0.815297,0.923423,0.440860,0.457286
2018 Mikko Rantanen,Mikko Rantanen,22,1,1,2019,74.0,1542.300000,31.0,56.0,34.0,...,6.822104,2.295273,1.624311,2.100759,1.488951,1.050379,0.974586,0.990654,0.352273,0.380952
2018 Artemi Panarin,Artemi Panarin,27,1,1,2019,79.0,1568.666667,28.0,59.0,44.0,...,7.881760,0.535487,0.650558,0.879728,0.850730,0.841479,0.850730,0.166667,0.318182,0.352459
2018 Sebastian Aho,Sebastian Aho,21,1,1,2019,82.0,1652.366667,30.0,53.0,29.0,...,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.928571,0.357143,0.392157
2018 Sebastian Aho,Sebastian Aho,21,1,1,2019,82.0,1652.366667,30.0,53.0,29.0,...,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.928571,0.357143,0.392157


In [100]:
xcols = ['forward', 'signing_age', 'signing_status'] + list(st.columns)[-198:]
xcols

['forward',
 'signing_age',
 'signing_status',
 'GP',
 'TOI',
 'Goals',
 'Total Assists',
 'First Assists',
 'Second Assists',
 'Total Points',
 'Shots',
 'iCF',
 'iFF',
 'iSCF',
 'iHDCF',
 'Rush Attempts',
 'Rebounds Created',
 'PIM',
 'Total Penalties',
 'Minor',
 'Major',
 'Misconduct',
 'Penalties Drawn',
 'Giveaways',
 'Takeaways',
 'Hits',
 'Hits Taken',
 'Shots Blocked',
 'Faceoffs Won',
 'Faceoffs Lost',
 'sum GP',
 'sum TOI',
 'sum Goals',
 'sum Total Assists',
 'sum First Assists',
 'sum Second Assists',
 'sum Total Points',
 'sum Shots',
 'sum iCF',
 'sum iFF',
 'sum iSCF',
 'sum iHDCF',
 'sum Rush Attempts',
 'sum Rebounds Created',
 'sum PIM',
 'sum Total Penalties',
 'sum Minor',
 'sum Major',
 'sum Misconduct',
 'sum Penalties Drawn',
 'sum Giveaways',
 'sum Takeaways',
 'sum Hits',
 'sum Hits Taken',
 'sum Shots Blocked',
 'sum Faceoffs Won',
 'sum Faceoffs Lost',
 'mean GP',
 'mean TOI',
 'mean Goals',
 'mean Total Assists',
 'mean First Assists',
 'mean Second Assists

In [94]:
list(st.columns)

['Player',
 'signing_age',
 'forward',
 'signing_status',
 'signing_year',
 'GP',
 'TOI',
 'Goals',
 'Total Assists',
 'First Assists',
 'Second Assists',
 'Total Points',
 'Shots',
 'iCF',
 'iFF',
 'iSCF',
 'iHDCF',
 'Rush Attempts',
 'Rebounds Created',
 'PIM',
 'Total Penalties',
 'Minor',
 'Major',
 'Misconduct',
 'Penalties Drawn',
 'Giveaways',
 'Takeaways',
 'Hits',
 'Hits Taken',
 'Shots Blocked',
 'Faceoffs Won',
 'Faceoffs Lost',
 'sum GP',
 'sum TOI',
 'sum Goals',
 'sum Total Assists',
 'sum First Assists',
 'sum Second Assists',
 'sum Total Points',
 'sum Shots',
 'sum iCF',
 'sum iFF',
 'sum iSCF',
 'sum iHDCF',
 'sum Rush Attempts',
 'sum Rebounds Created',
 'sum PIM',
 'sum Total Penalties',
 'sum Minor',
 'sum Major',
 'sum Misconduct',
 'sum Penalties Drawn',
 'sum Giveaways',
 'sum Takeaways',
 'sum Hits',
 'sum Hits Taken',
 'sum Shots Blocked',
 'sum Faceoffs Won',
 'sum Faceoffs Lost',
 'mean GP',
 'mean TOI',
 'mean Goals',
 'mean Total Assists',
 'mean First Ass

In [123]:
st.shape

(246, 203)

In [124]:
st.drop_duplicates(subset = ['Player', 'signing_age'], inplace=True)
st.shape

(245, 203)

In [113]:
len(st.Player.unique())

245

In [179]:
modst = dc.clean_features_data(sql=False, new_fas=False)

In [180]:
modst.shape

(1753, 224)

In [144]:
df = pd.read_csv('../data/up_all_pst.csv')
dfsummable = df.drop(['SH%', 'Faceoffs %', 'IPP'], axis=1)

summable_stats = ['GP', 'TOI', 'Goals',
            'Total Assists', 'First Assists', 'Second Assists', 'Total Points',
           'Shots', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
           'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
           'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
           'Hits Taken', 'Shots Blocked', 'Faceoffs Won', 'Faceoffs Lost']


#read in on_ice relative stats per season and over 3 year window
oirel = pd.read_csv('../data/up_all_oirel.csv')
woirel = pd.read_csv('../data/up_all_woirel.csv')


#fix column labeling
oirel['Season'] = oirel['Unnamed: 0.1']
oirel.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
woirel['Season'] = woirel['Unnamed: 0.1']
woirel.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)


#set indices to be season/player combo
oirel['Season_Player'] = oirel.Season.apply(str)
oirel.Season_Player = oirel.Season_Player + ' ' + oirel.Player
woirel['Season_Player'] = woirel.Season.apply(str)
woirel.Season_Player = woirel.Season_Player + ' ' + woirel.Player
oirel.set_index(oirel.Season_Player, inplace=True)
woirel.set_index(woirel.Season_Player, inplace=True)

In [145]:
contracts = dc.clean_contracts_data()

In [146]:
contracts.shape

(1762, 27)

In [149]:
len(contracts.Player.unique())

935

In [150]:
len(contracts.contract_id.unique())

1762

In [147]:
#set up default dictionaries to hold individual player stats
allstats = defaultdict(pd.DataFrame)
allsumstats = defaultdict(pd.DataFrame)
allmeanstats = defaultdict(pd.DataFrame)

In [148]:
#Give each player it's own data frame of stats all linked together in a dictionary with
#player names as keys
#Take season total stats and aggregate them over a 3 year window
for player in contracts['Player'].unique():
    allstats[player] = dfsummable[df.Player == player]
    allstats[player].sort_values(by='Season', inplace=True)
    allstats[player]['Season_index'] = pd.date_range(end='2019',
                                               periods = allstats[player].shape[0],
                                               freq='Y')
    allstats[player].set_index(allstats[player].Season_index, inplace=True)
    allsumstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=1)
                  .agg(np.sum))
    allmeanstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=1)
              .agg(np.mean))
    allmeanstats[player]['SH%'] = allsumstats[player]['Shots'] / allsumstats[player]['Goals']
    allmeanstats[player]['Faceoffs %'] = allsumstats[player]['Faceoffs Won'] / (
    allsumstats[player]['Faceoffs Won'] + allsumstats[player]['Faceoffs Lost'])

In [151]:
scols, mcols = [], []
#Relabeling sum and mean columns for clarity, arbitrarily using Pavelski for convenience,
#could be any player
for i in allsumstats['Joe Pavelski'].columns:
    scols.append('sum '+i)
for i in allmeanstats['Joe Pavelski'].columns:
    mcols.append('mean '+i)
for p in allstats:
    allsumstats[p].columns = scols
    allmeanstats[p].columns = mcols
    allstats[p] = pd.concat([allstats[p], allsumstats[p], allmeanstats[p]], axis=1)

In [152]:
len(allstats)

935

In [158]:
allsumstats['Tomas Hertl'].shape, allmeanstats['Tomas Hertl'].shape

((6, 27), (6, 29))

In [153]:
#get a combined dataframe with all relevant player/contract years
players = list(allstats.keys())
allallstats = pd.DataFrame()

for p in players:
    allallstats = pd.concat([allallstats, allstats[p].set_index(allstats[p].Season_Player)])


In [154]:
allallstats.shape

(5673, 89)

In [159]:
#drop duplicate column names
m_oirel = oirel.drop(['Player', 'Team', 'Position', 'GP', 'TOI', 'Season'], axis=1)
m_woirel = woirel.drop(['Player', 'Team', 'Position', 'GP', 'TOI', 'Season'], axis=1)

In [160]:
#relabel 3yr window columns for clarity
wcols = []
for i in m_woirel.columns:
    wcols.append('3yr ' + i)
m_woirel.columns = wcols

In [161]:
#merge on ice relative stats to the other stats
allallstats = pd.merge(allallstats, m_oirel, on = 'Season_Player')
allallstats = pd.merge(allallstats, m_woirel, on = 'Season_Player')

In [162]:
allallstats.shape

(5307, 182)

In [167]:
#remove contracts signed before 2010 (lack of stats)
table = contracts[contracts.signing_year > 2009]#.set_index('contract_id')


In [168]:
table.shape

(1753, 27)

In [169]:
#add column to line up contract years and stats years
table['year_match'] = table.signing_year - 1

#merge contracts and stats into a single table
table = pd.merge(table, allallstats,
            how = 'left', left_on = ['Player', 'year_match'],
            right_on = ['Player', 'Season'])
#drop rows that had missing seasons / indexing issues leading to NaNs
#1700 rows down to 1440 ---- 
#table.dropna(inplace=True)

In [175]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 564 entries, 4 to 1703
Columns: 209 entries, player_id to 3yr Season_Player
dtypes: bool(1), datetime64[ns](2), float64(148), int64(12), object(46)
memory usage: 921.5+ KB


In [447]:
modst = dc.clean_features_data(sql=False, new_fas=False)

In [462]:
reload(cr)
reload(dc)

<module 'data_cleaning.data_cleaning' from '/Users/colinshanahan/galvanize/nhl-contract-predictions/data_cleaning/data_cleaning.py'>

In [477]:
con_reg = cr.ContractRegressor()

In [478]:
X = con_reg.pick_cols(modst)
y = modst[['cap_pct', 'length']]
X.shape, y.shape

((1464, 197), (1464, 2))

In [479]:
X, y = con_reg.drop_duplicate_rows(X), con_reg.drop_duplicate_rows(y)

In [480]:
X = con_reg.impute(X)

In [481]:
split = (
    train_test_split(X, y['cap_pct'], y['length'], 
                     test_size = 0.15))

X_train, X_test, yp_train, yp_test, yl_train, yl_test = split

In [482]:
con_reg.fit(X_train, yp_train, yl_train)

In [483]:
con_reg.predict_global_mean(yp_train, yp_test, yl_train, yl_test)
print()
preds = con_reg.predict(X_test)
con_reg.score(X_test, yp_test, yl_test)

RMSE pick mean cap_pct: 3.06%
    translates to 2019 Cap Hit of: $2539800.0
RMSE pick mean length: 2.0 years

RMSE Cap_pct: 0.99%
    translates to 2019 Cap Hit of: $821700.0
RMSE Length: 1.2 years


(0.99, 821700.0, 1.2)

In [456]:
modst_newonly = dc.clean_features_data(sql=False, new_fas=False)

In [489]:
con_reg_newonly = cr.ContractRegressor()
Xn = con_reg_newonly.pick_cols(modst_newonly)
yn = modst_newonly[['cap_pct', 'length']]
Xn, yn = con_reg_newonly.drop_duplicate_rows(Xn), con_reg_newonly.drop_duplicate_rows(yn)
Xn = con_reg.impute(Xn)

In [490]:
split = (
    train_test_split(Xn, yn['cap_pct'], yn['length'], 
                     test_size = 0.15))

X_train, X_test, yp_train, yp_test, yl_train, yl_test = split

In [491]:
con_reg_newonly.fit(X_train, yp_train, yl_train)

In [492]:
con_reg_newonly.predict_global_mean(yp_train, yp_test, yl_train, yl_test)
print()
preds = con_reg_newonly.predict(X_test)
con_reg_newonly.score(X_test, yp_test, yl_test)

RMSE pick mean cap_pct: 2.73%
    translates to 2019 Cap Hit of: $2265900.0
RMSE pick mean length: 1.6 years

RMSE Cap_pct: 0.83%
    translates to 2019 Cap Hit of: $688900.0
RMSE Length: 1.0 years


(0.83, 688900.0, 1.0)

In [493]:
X.shape, y.shape

((1460, 197), (1460, 2))